In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf


import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf


In [2]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

In [3]:
var spark = SparkSession
.builder()
.appName("Java Spark SQL basic example")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@335ff79e
defined function sc

In [4]:
val frame = spark.read.option("header", true).option("delimiter", ",").option("inferSchema",true).csv("homeworkfile.csv")

frame: DataFrame = [country_code: string, device_id: string ... 12 more fields]

In [5]:
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}


import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@457ae07c
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSet}


In [10]:
frame.printSchema

root
 |-- country_code: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- advertising_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timezone: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- app: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- v: string (nullable = true)
 |-- version: string (nullable = true)



In [17]:
//2.Calculate daily active users trend over month
val newFrame = frame.withColumn("Date_of_event",to_date(col("timestamp"),"yyyy-MM-dd"))
newFrame.groupBy("Date_of_event").agg(count("device_id").alias("Number of active users")).sort("Date_of_event").show(100)

+-------------+----------------------+
|Date_of_event|Number of active users|
+-------------+----------------------+
|   2018-02-02|                 25455|
|   2018-02-03|                 26682|
|   2018-02-04|                 28148|
|   2018-02-05|                 25876|
|   2018-02-06|                 25070|
|   2018-02-07|                 24123|
|   2018-02-08|                 25137|
|   2018-02-09|                 25859|
|   2018-02-10|                 26963|
|   2018-02-11|                 28552|
|   2018-02-12|                 30262|
|   2018-02-13|                 26881|
|   2018-02-14|                 26451|
|   2018-02-15|                 25735|
|   2018-02-16|                 26747|
|   2018-02-17|                 26456|
|   2018-02-18|                 27925|
|   2018-02-19|                 25989|
|   2018-02-20|                 25734|
|   2018-02-21|                 26540|
|   2018-02-22|                 26768|
|   2018-02-23|                 27600|
|   2018-02-24|          

newFrame: DataFrame = [country_code: string, device_id: string ... 13 more fields]

In [64]:
// 3. Calculate weekly seasonality
var x = List(1,2,3,4,5,6,7)
var weekdayFrame = frame.withColumn("timestamp", to_timestamp(col("timestamp")))
  .withColumn("Day_of_week", date_format(col("timestamp"), "u"))
  .withColumn("Week_of_month", date_format(col("timestamp"), "W"))

var grouped_Weekday = weekdayFrame.cube("Week_of_month", "Day_of_week").agg(count("device_id").alias("Activity"))
var filtered_Weekday = grouped_Weekday.filter(col("Week_of_month").isin(x:_*)).filter(col("Day_of_week").isin(x:_*))
var sorted_Weekday = filtered_Weekday.sort("Day_of_week").sort("Week_of_month")

sorted_Weekday.show(50)
// As timestamp is from 02-02-2018 to 02-03-2018  2 fridays of first week are 
// included which brought to activity to be doubled
// Also I could not sort teh Day_of_wekk column

+-------------+-----------+--------+
|Week_of_month|Day_of_week|Activity|
+-------------+-----------+--------+
|            1|          6|   26682|
|            1|          4|   27719|
|            1|          5|   52733|
|            2|          7|   28148|
|            2|          4|   25137|
|            2|          5|   25859|
|            2|          6|   26963|
|            2|          3|   24123|
|            2|          2|   25070|
|            2|          1|   25876|
|            3|          5|   26747|
|            3|          3|   26451|
|            3|          1|   30262|
|            3|          2|   26881|
|            3|          6|   26456|
|            3|          4|   25735|
|            3|          7|   28552|
|            4|          5|   27600|
|            4|          2|   25734|
|            4|          6|   28744|
|            4|          4|   26768|
|            4|          1|   25989|
|            4|          7|   27925|
|            4|          3|   26540|
|

x: List[Int] = List(1, 2, 3, 4, 5, 6, 7)
weekdayFrame: DataFrame = [country_code: string, device_id: string ... 14 more fields]
grouped_Weekday: DataFrame = [Week_of_month: string, Day_of_week: string ... 1 more field]
filtered_Weekday: Dataset[Row] = [Week_of_month: string, Day_of_week: string ... 1 more field]
sorted_Weekday: Dataset[Row] = [Week_of_month: string, Day_of_week: string ... 1 more field]

In [45]:
/* Social Users description*/

var x = List[String]("photo_like","photo_open","photo_comment")
frame.groupBy("event_type").agg(count("device_id").alias("Number of active users"))
.sort(desc("Number of active users")).filter(col("event_type").isin(x:_*)).show()

// No comment at this moment :) just assuming that Social users are not greater than 6

+-------------+----------------------+
|   event_type|Number of active users|
+-------------+----------------------+
|   photo_open|                   609|
|   photo_like|                    91|
|photo_comment|                     6|
+-------------+----------------------+



x: List[String] = List("photo_like", "photo_open", "photo_comment")